Re-run cell with animated chart multiple times in Jupyter Notebook <br>
[Ref](https://stackoverflow.com/questions/35532498/animation-in-ipython-notebook)<br>
[Ref](https://discourse.jupyter.org/t/matplotlib-animation-not-appearing-in-jupyter-notebook/24938/5)


In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

# Re-run cell with animated chart multiple times in Jupyter Notebook
plt.rcParams["animation.html"] = "jshtml"
# plt.ioff()

In [29]:
# Get S&P 500 companies with theirs tickers: less stable but faster method
wiki_url = "https://en.wikipedia.org/wiki/List_of_S&P_500_companies"
# identify the table in the HTML by its unique id

sp500 = pd.read_html(wiki_url, attrs={"id": "constituents"})[0]
sp500.sort_values("Symbol", inplace=True)

In [3]:
shares_diluted_list = []
for ticker in sp500["Symbol"]:
    try:
        stock = yf.Ticker(ticker)
        shares_diluted = stock.quarterly_income_stmt.loc[["Diluted Average Shares"]]
    except KeyError:
        stock = yf.Ticker(ticker.replace(".", "-"))
        shares_diluted = stock.quarterly_income_stmt.loc[["Diluted Average Shares"]]
    shares_diluted = shares_diluted.transpose()
    shares_diluted["ticker"] = ticker
    shares_diluted_list.append(shares_diluted)

In [30]:
shares_diluted_df = pd.concat(shares_diluted_list)
shares_diluted_df.dropna(subset="Diluted Average Shares", inplace=True)
shares_diluted_df

,Diluted Average Shares,ticker
2024-07-31,291000000.0,A
2024-04-30,293000000.0,A
2024-01-31,294000000.0,A
2023-10-31,293000000.0,A
2023-07-31,295000000.0,A
...,...,...
2024-09-30,453500000.0,ZTS
2024-06-30,456000000.0,ZTS
2024-03-31,458800000.0,ZTS
2023-12-31,460100000.0,ZTS


In [31]:
frames = shares_diluted_df.index.unique()

In [68]:
# Format for plotting with seaborn.objects
shares_diluted_df["Diluted Average Shares"] = shares_diluted_df[
    "Diluted Average Shares"
].astype("int")

In [116]:
import textalloc as ta


def create_animation(frame):
    ax.clear()  # delete previous axes when re-run cell
    shares_diluted_frame = shares_diluted_df.loc[shares_diluted_df.index == frame]
    shares_top10 = shares_diluted_frame.sort_values(
        "Diluted Average Shares", ascending=False
    ).head(10)

    shares_top10.reset_index(inplace=True)
    sns.barplot(
        shares_top10,
        x="Diluted Average Shares",
        y="ticker",
        ax=ax,
        hue="ticker",
        palette="viridis",
    )
    ta.allocate(
        ax,
        shares_top10["Diluted Average Shares"],
        shares_top10.index + 0.3,
        [
            f"{annotation/10**6:,.0f}M"
            for annotation in shares_top10["Diluted Average Shares"]
        ],
        draw_lines=False,
    )

In [117]:
fig, ax = plt.subplots()
# Avoid naming animated chart as "animation" to not be confusing with animation.html
animated_chart = animation.FuncAnimation(
    fig, create_animation, frames=frames, interval=500
)
plt.close()  # delete previous figs when re-run cell
animated_chart

**Workaround for formatting annotation in `seaborn.objects`** <br>
[Ref](https://stackoverflow.com/questions/76812518/formatting-text-labels-in-seaborn-objects)


import plotly.express as px

px.bar(
df, x="ticker", y="Diluted Average Shares", color="ticker", animation_frame="index"
)
